In [1]:
#The estimation of the MLE is based on the material from Quantecon

In [2]:
#First, we need the following imports

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size
import numpy as np
from numpy import exp
from scipy.special import factorial
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
from statsmodels.api import Poisson
from scipy import stats
from scipy.stats import norm
from statsmodels.iolib.summary2 import summary_col

SyntaxError: invalid syntax (<ipython-input-8-6c4855eabb09>, line 4)

In [22]:
%%html
<p>
Consider the probit model where the dependent variable ($Y$) is binary, and the independent variables are given by the vector $X$. 
In addition, assume that $Pr(Y=1|X)= Φ(X^T β)$ where $Φ$ is the CDF of the standard normal distribution. 
</p>

<p>
The objective is to maximize the likelihood function over the parameters to find the Maximum Likelihood Estimator (MLE). 
To find the MLE estimator 
</p>

<p>
$$
\ln L(\beta \mid X, Y)=\sum_{i=1}^n\left[y_i \ln \Phi\left(x_i^{\prime} \beta\right)+\left(1-y_i\right) \ln \left(1-\Phi\left(x_i^{\prime} \beta\right)\right)\right]
$$
</p>

<p>
The basic probit model takes a binary dependent variable, Y, and assumes that Pr(Y=1|X)=Φ(XTβ) where X is the matrix of independent variables, β is the vector of parameters to estimate, and Φ is the CDF of the standard normal distribution. We want to take the likelihood function L=PR(β|X,Y), and maximize it over β to get the most likely β paramaters given the data X,Y. We usually use the log of the likelihood function in practice, because it is simpler in both math and computation, and the maximum point is the same. The probit log likelihood is as follows:
</p>

%%latex

$$
\ln L(\beta \mid X, Y)=\sum_{i=1}^n\left[y_i \ln \Phi\left(x_i^{\prime} \beta\right)+\left(1-y_i\right) \ln \left(1-\Phi\left(x_i^{\prime} \beta\right)\right)\right]
$$
